### 精度对齐

In [1]:
import os
import re
import numpy as np
import sys

def load_txt(file):
    data = np.loadtxt(file)
    return data

def read_chgcar_density(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    # VASP 文件的头部信息
    scale_factor = float(lines[1].strip())
    # 读取晶格矢量
    lattice_vectors = np.array([list(map(float, line.split())) for line in lines[2:5]])
    # 读取原子类型和数量
    atom_types = lines[5].split()
    atom_counts = list(map(int, lines[6].split()))
    total_atoms = sum(atom_counts)
    # 读取密度数据的起始行
    density_start_line = 9 + total_atoms
    # 读取密度网格信息
    grid_size = list(map(int, lines[density_start_line].split()))
    nx, ny, nz = grid_size
    # 读取密度数据
    density_data = []
    for line in lines[density_start_line + 1:]:
        density_data.extend(map(float, line.split()))
    # 将密度数据转换为三维数组
    density_array = np.array(density_data)
    return density_array

def extract_float_from_text(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    floats = re.findall(r'\d+\.\d+', text)
    floats = [float(f) for f in floats]
    return floats[:-3]

def compute_error(data1, data2):
    # compute the mae and relative mae
    mae = np.mean(np.abs(data1 - data2))
    rmae = mae / np.mean(np.abs(data1))
    return mae, rmae

def list_files(path, cmp_path):
    chg_atoms = [r'^C_charges.*', r'^H_charges.*', r'^N_charges.*', r'^O_charges.*', r'^DOS.*']
    tf_path = os.path.join('./tutorials', path)
    ms_path = os.path.join(cmp_path, path)

    files = os.listdir(tf_path)
    for chg_atom in chg_atoms:
        for file in files:
            if re.match(chg_atom, file):
                tf_chg = load_txt(os.path.join(tf_path, file))
                ms_chg = load_txt(os.path.join(ms_path, file))
                mae, rmae = compute_error(tf_chg, ms_chg)
                print(f'[{chg_atom[1:4]}] {file.split("_")[-1].split(".")[0][:10]:10}: MAE={mae:2.8f}, RMAE={rmae:2.8f}')
    
    for file in files:
        if re.match(r'^Pred_CHG.*', file):
            tf_chg = read_chgcar_density(os.path.join(tf_path, file))
            ms_chg = read_chgcar_density(os.path.join(ms_path, file))
            mae, rmae = compute_error(tf_chg, ms_chg)
            print(f'[CHG] {file.split("_")[-1].split(".")[0][:10]:10}: MAE={mae:2.8f}, RMAE={rmae:2.8f}')

    if path[:2] == 'T1':
        tf_floats = extract_float_from_text(os.path.join(tf_path, 'OUT_DATA.log'))
        ms_floats = extract_float_from_text(os.path.join(ms_path, 'OUT_DATA.log'))
        mae, rmae = compute_error(np.array(tf_floats), np.array(ms_floats))
        print(f'[OUT]           : MAE={mae:2.8f}, RMAE={rmae:2.8f}')

In [2]:
orig_stdout = sys.stdout
f = open('KerasVSpynative.log', 'w')
sys.stdout = f

paths = [
    "T1_Predicting_CHG_E_DOS",
    "T1_Predicting_CHG_E_DOS_ALL",
    "T2_Train_EFP",
    "T3_Train_DOS"
]

for path in paths:
    print("========================================{:30}========================================".format(path))
    list_files(path, '../ML-DFT-ms39/PYNATIVE/tutorials')
    print("\n")

sys.stdout = orig_stdout
f.close()

In [2]:
orig_stdout = sys.stdout
f = open('KerasVSgraph.log', 'w')
sys.stdout = f

paths = [
    "T1_Predicting_CHG_E_DOS",
    "T1_Predicting_CHG_E_DOS_ALL",
    "T2_Train_EFP",
    "T3_Train_DOS"
]

for path in paths:
    print("========================================{:30}========================================".format(path))
    list_files(path, '../ML-DFT-ms39/GRAPH/tutorials')
    print("\n")

sys.stdout = orig_stdout
f.close()

In [6]:
import re
import numpy as np

def extract_floats_from_text(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    
    # 使用正则表达式匹配所有浮点数（包含小数点的数字）
    floats = re.findall(r'\d+\.\d+', text)
    
    # 将匹配的数字转换为浮点数
    floats = [float(num) for num in floats]
    
    return floats

# 示例使用
file_path = './tutorials/T1_Predicting_CHG_E_DOS/OUT_DATA'
floats_array = extract_floats_from_text(file_path)
floats_array = np.array(floats_array)
print(floats_array)


[4.00543830e+00 4.00825170e+00 3.99982380e+00 4.00722270e+00
 4.00700140e+00 4.00096270e+00 9.82319240e-01 9.81335640e-01
 9.81985330e-01 9.82557300e-01 9.83064800e-01 9.82784100e-01
 7.57137051e+01 5.58588450e-01 4.59736620e-01 2.81325550e-01
 1.30061450e+00 6.47097350e-01 4.92146000e-01 2.02789740e+00
 4.86163600e-01 1.04766420e+00 1.89562510e+00 1.98968280e+00
 6.15723000e-01 3.10548110e+00 7.44978850e-01 1.51498580e+00
 1.27760160e+00 9.15084400e-02 5.36115200e-01 2.62290450e-01
 5.61034000e-01 2.86039860e-01 7.69326270e-01 3.73211500e-01
 2.89164750e-01 2.39248200e-01 1.54484400e-01 1.15564200e-01
 1.98507770e-01 2.39390160e-01 6.90811400e-02 1.30469820e+00
 1.01904190e+00 1.70750920e-01 1.28440010e-01 2.21347560e-01
 3.46651600e-02 4.62489650e+00 4.33167270e+00 1.68412070e+00
 7.38772000e-01 4.18175580e-01 2.27746300e+00 6.23646688e+00
 1.57894570e-02 1.33203185e+00 1.65031250e-02 4.90443504e+00
 2.45868170e-02 2.99227392e+01 1.27083724e+00 1.17011126e+02]


### COV

In [2]:
from keras.layers import Conv1D, Input
import numpy as np
from keras.models import Model

def test_model():
    input = Input(shape=(3, 1))
    cov = Conv1D(2,kernel_size=2,activation='relu')(input)
    net = Model(inputs=input, outputs=cov)
    return net

net = test_model()
print(net)
random_input = np.random.rand(1,3,1)
output = net.predict(random_input)
print(random_input)
print(net.layers[1].get_weights())
print(output)
print(random_input.shape)
print(net.layers[1].get_weights()[0].shape, net.layers[1].get_weights()[1].shape)
print(output.shape)

[[[0.06014159]
  [0.64984425]
  [0.36385862]]]
[array([[[-0.03125644,  0.09805012]],

       [[ 0.33049035,  0.12989593]]], dtype=float32), array([0., 0.], dtype=float32)]
[[[0.21288744 0.09030901]
  [0.09993994 0.11098105]]]
(1, 3, 1)
(2, 1, 2) (2,)
(1, 2, 2)


### DOS

In [5]:
import numpy as np
import keras
from DOS import init_DOSmod
import tensorflow as tf

net = init_DOSmod(10)
net.load_weights('../Trained_models/weights_DOS.hdf5')

input1 = np.linspace(1,2,700*10).reshape(10,700).astype(np.float32)
input2 = np.linspace(1,2,568*10).reshape(10,568).astype(np.float32)
input3 = np.linspace(1,2,700*10).reshape(10,700).astype(np.float32)
input4 = np.linspace(1,2,700*10).reshape(10,700).astype(np.float32)
input5 = np.array([1]).astype(np.float32)
input6 = np.linspace(1,2,341*10).reshape(10,341).astype(np.float32)
input7 = np.linspace(1,2,341*10).reshape(10,341).astype(np.float32)
input8 = np.linspace(1,2,341*10).reshape(10,341).astype(np.float32)
input9 = np.linspace(1,2,341*10).reshape(10,341).astype(np.float32)
print(input1[0,:2], input2[0,:2], input3[0,:2], input4[0,:2], input5, input6[0,:2], input7[0,:2], input8[0,:2], input9[0,:2])

inputs = [input1, input2, input3, input4, input5, input6, input7, input8, input9]
inputs = [np.expand_dims(i, axis=0) for i in inputs]

resultD = []
resultvbcb=[]
for i in range(1):
    Pred,vbcb=net.predict(inputs, batch_size=1)
    resultD.append(Pred*inputs[4])
    resultvbcb.append(vbcb)
resultD=np.array(resultD)
Pred = resultD.mean(axis=0)
resultvbcb=np.array(resultvbcb)
devVB=resultvbcb.std(axis=0)[0][0]
devCB=resultvbcb.std(axis=0)[0][1]
Pred_vb =(-1)*resultvbcb.mean(axis=0)[0][0]
Pred_cb =(-1)*resultvbcb.mean(axis=0)[0][1]
uncertainty = resultD.std(axis=0)
uncertainty=np.squeeze(uncertainty)
resultvbcb=np.vstack(resultvbcb)
Bandgap=Pred_cb-Pred_vb
devBG=resultvbcb[:,0]-resultvbcb[:,1]
devBG=devBG.std(axis=0)

print(Pred.shape, uncertainty.shape, Pred_vb, devVB, Pred_cb, devCB, Bandgap, devBG)
print(Pred[0,:5], Pred[0,-5:], np.min(Pred), np.max(Pred), sep='\n')

#print(Pred, uncertainty,Pred_vb,devVB,Pred_cb,devCB,Bandgap,devBG)
# print(Pred[0,:5], uncertainty[:5], Pred_vb, devVB, Pred_cb, devCB, Bandgap, devBG, sep='\n')
# print(np.min(Pred), np.max(Pred), np.min(uncertainty), np.max(uncertainty), sep='\n')

[1.        1.0001429] [1.        1.0001761] [1.        1.0001429] [1.        1.0001429] [1.] [1.        1.0002934] [1.        1.0002934] [1.        1.0002934] [1.        1.0002934]
(1, 341) (341,) -0.0 0.0 -54.09758377075195 0.0 -54.09758377075195 0.0
[0.         0.         0.         0.         0.03025112]
[118.314804 164.45445  201.31795  165.51218  113.63244 ]
0.0
307.36807


In [2]:
import tensorflow as tf
import keras.backend as K
from keras.models import Model
from keras.layers import Lambda,Input, concatenate,Dense,Reshape, Conv1D,Activation, Add, ReLU, TimeDistributed,Dropout
from keras.models import load_model
from keras.regularizers import l2,l1
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.optimizers import SGD,Adam
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
import numpy as np

padding_size=10

def init_DOSmod(padding_size):
    def dos_model():
        def single_atom_modelC_1():
            model_input=Input(shape=(700,))
            model_out=Dense(600,activation='relu',activity_regularizer=l2(0.1),kernel_initializer='glorot_uniform')(model_input)
            model_out=Dense(600,activation='relu',activity_regularizer=l2(0.1),kernel_initializer='glorot_uniform')(model_out)
            model_out=Dense(600,activation='relu',activity_regularizer=l2(0.1),kernel_initializer='glorot_uniform')(model_out)
            model_out=Dense(600,activation='relu',activity_regularizer=l2(0.1),kernel_initializer='glorot_uniform')(model_out)
            model_out=Dense(343,activation='relu',activity_regularizer=l2(0.1),kernel_initializer='glorot_uniform')(model_out)
            model_out=Reshape((343,-1))(model_out)
            model_out=Conv1D(3,kernel_size=3,activation='relu')(model_out)
            model_out=Lambda(lambda x: tf.keras.backend.mean(x,axis=-1))(model_out)
            model=Model(inputs=model_input, outputs=model_out)
            return model
        def single_atom_modelH_1():
            model_input=Input(shape=(568,))
            model_out=Dense(600,activation='relu',activity_regularizer=l2(0.1),kernel_initializer='glorot_uniform')(model_input)
            model_out=Dense(600,activation='relu',activity_regularizer=l2(0.1),kernel_initializer='glorot_uniform')(model_out)
            model_out=Dense(600,activation='relu',activity_regularizer=l2(0.1),kernel_initializer='glorot_uniform')(model_out)
            model_out=Dense(600,activation='relu',activity_regularizer=l2(0.1),kernel_initializer='glorot_uniform')(model_out)
            model_out=Dense(343,activation='relu',activity_regularizer=l2(0.1),kernel_initializer='glorot_uniform')(model_out)
            model_out=Reshape((343,-1))(model_out)
            model_out=Conv1D(3,kernel_size=3,activation='relu')(model_out)
            model_out=Lambda(lambda x: tf.keras.backend.mean(x,axis=-1))(model_out)
            model=Model(inputs=model_input, outputs=model_out)
            return model
        def single_atom_modelN_1():
            model_input=Input(shape=(700,))
            model_out=Dense(600,activation='relu',activity_regularizer=l2(0.1),kernel_initializer='glorot_uniform')(model_input)
            model_out=Dense(600,activation='relu',activity_regularizer=l2(0.1),kernel_initializer='glorot_uniform')(model_out)
            model_out=Dense(600,activation='relu',activity_regularizer=l2(0.1),kernel_initializer='glorot_uniform')(model_out)
            model_out=Dense(600,activation='relu',activity_regularizer=l2(0.1),kernel_initializer='glorot_uniform')(model_out)
            model_out=Dense(343,activation='relu',activity_regularizer=l2(0.1),kernel_initializer='glorot_uniform')(model_out)
            model_out=Reshape((343,-1))(model_out)
            model_out=Conv1D(3,kernel_size=3,activation='relu')(model_out)
            model_out=Lambda(lambda x: tf.keras.backend.mean(x,axis=-1))(model_out)
            model=Model(inputs=model_input, outputs=model_out)
            return model
        def single_atom_modelO_1():
            model_input=Input(shape=(700,))
            model_out=Dense(600,activation='relu',activity_regularizer=l2(0.1),kernel_initializer='glorot_uniform')(model_input)
            model_out=Dense(600,activation='relu',activity_regularizer=l2(0.1),kernel_initializer='glorot_uniform')(model_out)
            model_out=Dense(600,activation='relu',activity_regularizer=l2(0.1),kernel_initializer='glorot_uniform')(model_out)
            model_out=Dense(600,activation='relu',activity_regularizer=l2(0.1),kernel_initializer='glorot_uniform')(model_out)
            model_out=Dense(343,activation='relu',activity_regularizer=l2(0.1),kernel_initializer='glorot_uniform')(model_out)
            model_out=Reshape((343,-1))(model_out)
            model_out=Conv1D(3,kernel_size=3,activation='relu')(model_out)
            model_out=Lambda(lambda x: tf.keras.backend.mean(x,axis=-1))(model_out)
            model=Model(inputs=model_input, outputs=model_out)
            return model
        input1=Input(shape=(padding_size,700))
        input2=Input(shape=(padding_size,568))
        input3=Input(shape=(padding_size,700))
        input4=Input(shape=(padding_size,700))
        input5=Input(shape=(1,))
        input6=Input(shape=(padding_size,341))
        input7=Input(shape=(padding_size,341))
        input8=Input(shape=(padding_size,341))
        input9=Input(shape=(padding_size,341))
        model_out_C1=TimeDistributed(single_atom_modelC_1(),name='atom_dosC_1')(input1)
        model_out_H1=TimeDistributed(single_atom_modelH_1(),name='atom_dosH_1')(input2)
        model_out_N1=TimeDistributed(single_atom_modelN_1(),name='atom_dosN_1')(input3)
        model_out_O1=TimeDistributed(single_atom_modelO_1(),name='atom_dosO_1')(input4)
        D_C=Lambda(lambda x: tf.math.multiply(x[0],x[1]),name='D_C')([input6,model_out_C1])
        D_H=Lambda(lambda x: tf.math.multiply(x[0],x[1]),name='D_H')([input7,model_out_H1])
        D_N=Lambda(lambda x: tf.math.multiply(x[0],x[1]),name='D_N')([input8,model_out_N1])
        D_O=Lambda(lambda x: tf.math.multiply(x[0],x[1]),name='D_O')([input9,model_out_O1])
        model_added1=Add()([D_C,D_H,D_N,D_O])

        model_s1=Lambda(lambda x: tf.keras.backend.sum(x,axis=1))(model_added1)
        model_dos=Lambda(lambda x: x/input5,name='DOS1')(model_s1)
        bands=Dense(100,activation='relu',activity_regularizer=l2(0.01))(model_dos)
        bands=Dense(100,activation='relu',activity_regularizer=l2(0.01))(bands)
        bands=Dense(2,activation='relu')(bands)
        model= Model(inputs=[input1,input2,input3,input4,input5,input6,input7,input8,input9], outputs=[model_dos,bands])
        opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999)
        model.compile(loss="mean_squared_error",optimizer=opt,loss_weights=[1000,1])

        return model
    modelDOS=dos_model()
    return modelDOS

model = init_DOSmod(padding_size)

In [37]:
input1 = np.linspace(1,2,700*10).reshape(10,700).astype(np.float32)
input2 = np.linspace(1,2,568*10).reshape(10,568).astype(np.float32)
input3 = np.linspace(1,2,700*10).reshape(10,700).astype(np.float32)
input4 = np.linspace(1,2,700*10).reshape(10,700).astype(np.float32)
input5 = np.array([1]).astype(np.float32)
input6 = np.linspace(1,2,341*10).reshape(10,341).astype(np.float32)
input7 = np.linspace(1,2,341*10).reshape(10,341).astype(np.float32)
input8 = np.linspace(1,2,341*10).reshape(10,341).astype(np.float32)
input9 = np.linspace(1,2,341*10).reshape(10,341).astype(np.float32)
print(input1[0,:2], input2[0,:2], input3[0,:2], input4[0,:2], input5, input6[0,:2], input7[0,:2], input8[0,:2], input9[0,:2])

inputs = [input1, input2, input3, input4, input5, input6, input7, input8, input9]
inputs = [np.expand_dims(i, axis=0) for i in inputs]
output = model.predict(inputs, batch_size=1)
print(output)
# save model
model.save_weights('./model.hdf5')

[1.        1.0001429] [1.        1.0001761] [1.        1.0001429] [1.        1.0001429] [1.] [1.        1.0002934] [1.        1.0002934] [1.        1.0002934] [1.        1.0002934]
[array([[ 7.92782   ,  3.392028  ,  1.8162076 ,  6.0736217 ,  4.697298  ,
         3.3485682 ,  7.9982214 ,  4.410218  ,  6.67941   ,  5.3119183 ,
         6.7439604 ,  1.4122567 ,  9.527989  ,  2.6138964 ,  7.508136  ,
         0.7062812 ,  2.3905354 ,  2.6121025 ,  5.6294293 ,  2.1034234 ,
         3.0660615 ,  3.2437024 ,  3.115258  ,  1.3584677 ,  2.2491052 ,
         1.0826668 ,  2.6484418 ,  1.9205368 ,  3.6130838 ,  7.411451  ,
         5.312133  ,  8.841053  ,  6.206501  ,  3.373712  ,  3.9756603 ,
         4.765744  ,  3.8590481 ,  9.224546  ,  6.767336  ,  3.533239  ,
         3.7462835 ,  0.9238925 ,  0.21277952,  2.950078  ,  2.5659664 ,
         4.6525583 ,  5.1969876 ,  9.347906  ,  4.959262  ,  3.6720457 ,
         3.9378092 ,  4.858216  ,  3.7078404 ,  4.5242176 ,  3.3074877 ,
         1.8317

### CHG

In [6]:
import numpy as np
import keras
from CHG import init_chgmod
import tensorflow as tf

net = init_chgmod(10)
net.load_weights('../Trained_models/weights_CHG.hdf5')

input1 = np.linspace(0,1,10*360).reshape(10,360).astype(np.float32)
input2 = np.linspace(0,1,10*360).reshape(10,360).astype(np.float32)
input3 = np.linspace(0,1,10*360).reshape(10,360).astype(np.float32)
input4 = np.linspace(0,1,10*360).reshape(10,360).astype(np.float32)

inputs = [input1, input2, input3, input4]
inputs = [np.expand_dims(i, axis=0) for i in inputs]
outputs = net.predict(inputs)
print(outputs[0].shape, outputs[1].shape, outputs[2].shape, outputs[3].shape)
print(outputs[0][0,0,:5], outputs[1][0,0,:5], outputs[2][0,0,:5], outputs[3][0,0,:5])

(1, 10, 340) (1, 10, 208) (1, 10, 340) (1, 10, 340)
[2.7545054 0.4031816 4.274326  2.321104  4.463279 ] [3.5862846 4.52964   2.848134  3.7696545 4.4418006] [3.0112755 2.9462447 3.1302595 6.373972  3.9518328] [3.1376822 6.75171   7.038846  5.9741545 6.4981647]


### EFP

In [3]:
import numpy as np
import keras
from Energy import init_Emod
import tensorflow as tf

net = init_Emod(10)
net.load_weights('../Trained_models/weights_EFP.hdf5')

input1 = np.linspace(0,1,10*709).reshape(10,709).astype(np.float32)
input2 = np.linspace(0,1,10*577).reshape(10,577).astype(np.float32)
input3 = np.linspace(0,1,10*709).reshape(10,709).astype(np.float32)
input4 = np.linspace(0,1,10*709).reshape(10,709).astype(np.float32)
input5 = np.array([1]).astype(np.float32)
input6 = np.linspace(0,1,10*1).reshape(10,1).astype(np.float32)
input7 = np.linspace(0,1,10*1).reshape(10,1).astype(np.float32)
input8 = np.linspace(0,1,10*1).reshape(10,1).astype(np.float32)
input9 = np.linspace(0,1,10*1).reshape(10,1).astype(np.float32)

inputs = [input1, input2, input3, input4, input5, input6, input7, input8, input9]
inputs = [np.expand_dims(i, axis=0) for i in inputs]
outputs = net.predict(inputs)
out1,out2,out3,out4,out5,out6 = outputs
print(out1.shape, out2.shape, out3.shape, out4.shape, out5.shape, out6.shape)
print(out1[0], out2[0,0], out3[0,0], out4[0,0], out5[0,0], out6[0], sep='\n')

(1, 1) (1, 10, 3) (1, 10, 3) (1, 10, 3) (1, 10, 3) (1, 6)
[98.10155]
[-1.1681886 -1.1731949 -1.178201 ]
[-0.04971177 -0.04997553 -0.05023931]
[0.10500497 0.10545186 0.10589875]
[0.36152738 0.36307415 0.36462092]
[18721.4   18744.576 18767.768 18732.984 18756.168 18744.566]


### DOS

In [2]:
from keras.layers import Input, Dense, Conv1D, Reshape, Lambda, Dropout
from keras.models import Model
from keras.regularizers import l2
import tensorflow as tf

def single_atom_modelC_1():
    model_input=Input(shape=(700,))
    model_out=Dense(600,activation='relu',activity_regularizer=l2(0.1),kernel_initializer='glorot_uniform')(model_input)
    model_out=Dropout(0.1)(model_out,training=True)
    model_out=Dense(600,activation='relu',activity_regularizer=l2(0.1),kernel_initializer='glorot_uniform')(model_out)
    model_out=Dropout(0.1)(model_out,training=True)
    model_out=Dense(600,activation='relu',activity_regularizer=l2(0.1),kernel_initializer='glorot_uniform')(model_out)
    model_out=Dropout(0.1)(model_out,training=True)
    model_out=Dense(600,activation='relu',activity_regularizer=l2(0.1),kernel_initializer='glorot_uniform')(model_out)
    model_out=Dropout(0.1)(model_out,training=True)
    model_out=Dense(343,activation='relu',activity_regularizer=l2(0.1),kernel_initializer='glorot_uniform')(model_out)
    model_out=Reshape((343,-1))(model_out)
    model_out=Conv1D(3,kernel_size=3,activation='relu')(model_out)
    model_out=Lambda(lambda x: tf.keras.backend.mean(x,axis=-1))(model_out)
    model=Model(inputs=model_input, outputs=model_out)
    model.summary()
    return model

model = single_atom_modelC_1()




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 700)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 600)               420600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 600)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 600)               360600    
_________________________________________________________________
dropout_2 (Dropout)          (None, 600)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 600)               360600    
________________________

In [1]:
import h5py

In [5]:
filepath="../Trained_models/weights_DOS.hdf5"
with h5py.File(filepath, "r") as weights:
    print("Keys: %s" % weights.keys())
    # Keys: <KeysViewHDF5 ['DOS1', 'D_C', 'D_H', 'D_N', 'D_O', 'add_1', 'atom_dosC_1', 'atom_dosH_1', 'atom_dosN_1', 'atom_dosO_1', 'dense_41', 'dense_42', 'dense_43', 
    # 'input_10', 'input_11', 'input_12', 'input_14', 'input_15', 'input_16', 'input_17', 'input_9', 'lambda_13']>
    print("Keys: %s" % weights['DOS1'].keys())
    print("Keys: %s" % weights['D_C'].keys())
    print("Keys: %s" % weights['D_H'].keys())
    print("Keys: %s" % weights['D_N'].keys())
    print("Keys: %s" % weights['D_O'].keys())
    print("Keys: %s" % weights['add_1'].keys())
    print("Keys: %s" % weights['atom_dosC_1'].keys())
    print("Keys: %s" % weights['atom_dosH_1'].keys())
    print("Keys: %s" % weights['atom_dosN_1'].keys())
    print("Keys: %s" % weights['atom_dosO_1'].keys())
    print("Keys: %s" % weights['dense_41'].keys())
    print("Keys: %s" % weights['dense_42'].keys())
    print("Keys: %s" % weights['dense_43'].keys())
    print("Keys: %s" % weights['input_10'].keys())
    print("Keys: %s" % weights['input_11'].keys())
    print("Keys: %s" % weights['input_12'].keys())
    print("Keys: %s" % weights['input_14'].keys())
    print("Keys: %s" % weights['input_15'].keys())
    print("Keys: %s" % weights['input_16'].keys())
    print("Keys: %s" % weights['input_17'].keys())
    print("Keys: %s" % weights['input_9'].keys())
    print("Keys: %s" % weights['lambda_13'].keys())
    print("Keys: %s" % weights['atom_dosC_1']['conv1d_1'].keys())
    print("Keys: %s" % weights['atom_dosC_1']['conv1d_1']['bias:0'])
    print("Keys: %s" % weights['atom_dosC_1']['conv1d_1']['kernel:0'])

    # optimizer_weights = f['optimizer_weights']
    # print("Keys: %s" % optimizer_weights.keys())

Keys: <KeysViewHDF5 ['DOS1', 'D_C', 'D_H', 'D_N', 'D_O', 'add_1', 'atom_dosC_1', 'atom_dosH_1', 'atom_dosN_1', 'atom_dosO_1', 'dense_41', 'dense_42', 'dense_43', 'input_10', 'input_11', 'input_12', 'input_14', 'input_15', 'input_16', 'input_17', 'input_9', 'lambda_13']>
Keys: <KeysViewHDF5 []>
Keys: <KeysViewHDF5 []>
Keys: <KeysViewHDF5 []>
Keys: <KeysViewHDF5 []>
Keys: <KeysViewHDF5 []>
Keys: <KeysViewHDF5 []>
Keys: <KeysViewHDF5 ['conv1d_1', 'dense_21', 'dense_22', 'dense_23', 'dense_24', 'dense_25']>
Keys: <KeysViewHDF5 ['conv1d_2', 'dense_26', 'dense_27', 'dense_28', 'dense_29', 'dense_30']>
Keys: <KeysViewHDF5 ['conv1d_3', 'dense_31', 'dense_32', 'dense_33', 'dense_34', 'dense_35']>
Keys: <KeysViewHDF5 ['conv1d_4', 'dense_36', 'dense_37', 'dense_38', 'dense_39', 'dense_40']>
Keys: <KeysViewHDF5 ['dense_41']>
Keys: <KeysViewHDF5 ['dense_42']>
Keys: <KeysViewHDF5 ['dense_43']>
Keys: <KeysViewHDF5 []>
Keys: <KeysViewHDF5 []>
Keys: <KeysViewHDF5 []>
Keys: <KeysViewHDF5 []>
Keys: <Keys

In [1]:
import os
import numpy as np
from DOS import init_DOSmod

CONFIG_PATH1 = '../Trained_models/weights_DOS.hdf5'

padding_size = 10
rtmodel = init_DOSmod(padding_size)
rtmodel.load_weights(CONFIG_PATH1)
rtmodel.summary()

Using TensorFlow backend.





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 10, 700)      0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 10, 568)      0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 10, 700)      0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 10, 700)      0                                 

In [2]:
rtmodel.D_C.summary()

AttributeError: 'Model' object has no attribute 'D_C'

In [ ]:
filepath="../Trained_models/weights_CHG.hdf5"
with h5py.File(filepath, "r") as f:
    print("Keys: %s" % f.keys())
    # Keys: <KeysViewHDF5 ['model_weights', 'optimizer_weights']>
    weights = f['model_weights']
    print("Keys: %s" % weights.keys())
    # Keys: <KeysViewHDF5 ['add_10', 'add_11', 'add_9', 'atomC1_chg', 'atomH1_chg', 'atomN1_chg', 'atomO1_chg', 'input_1', 'input_2', 'input_3', 'input_4', 'lambda_640', 'lambda_641', 'lambda_642', 'lambda_643']>
    print("Keys: %s" % weights['add_10'].keys())
    print("Keys: %s" % weights['add_11'].keys())
    print("Keys: %s" % weights['add_9'].keys())
    print("Keys: %s" % weights['atomC1_chg'].keys())
    print("Keys: %s" % weights['atomH1_chg'].keys())
    print("Keys: %s" % weights['atomN1_chg'].keys())
    print("Keys: %s" % weights['atomO1_chg'].keys())
    print("Keys: %s" % weights['input_1'].keys())
    print("Keys: %s" % weights['input_2'].keys())
    print("Keys: %s" % weights['input_3'].keys())
    print("Keys: %s" % weights['input_4'].keys())
    print("Keys: %s" % weights['lambda_640'].keys())
    print("Keys: %s" % weights['lambda_641'].keys())
    print("Keys: %s" % weights['lambda_642'].keys())
    print("Keys: %s" % weights['lambda_643'].keys())
    print("Keys: %s" % weights['dense_22'].keys())

    # optimizer_weights = f['optimizer_weights']
    # print("Keys: %s" % optimizer_weights.keys())

### 查看网络输出

In [1]:
import numpy as np
import keras.backend as K
from keras import Model
from keras.layers import *
class Normal(Layer):
    def __init__(self, **kwargs):
        super(Normal, self).__init__(**kwargs) #必须定义的
    def build(self, input_shape):
        # 添加可训练的参数
        self.kernel = self.add_weight(name='kernel',shape=(1,),
                                      initializer='zeros',trainable=True)
        self.built = True
    def call(self, x):
        # 定义功能，相当于Lambda层的功能函数
        self.x_normalized = K.l2_normalize(x, -1)
        return self.x_normalized * self.kernel

x_in = Input(shape=(784,))
x = x_in

x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
normal = Normal()
x = normal(x)
x = Dense(10, activation='softmax')(x)

model = Model(x_in, x)

Using TensorFlow backend.
/home/cenjianhuan/miniconda3/envs/MLDFT37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/cenjianhuan/miniconda3/envs/MLDFT37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/cenjianhuan/miniconda3/envs/MLDFT37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/home/cenjianhuan/miniconda3/envs/MLDFT37/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/cenjianhuan/miniconda3/envs/MLDFT37/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/cenjianhuan/miniconda3/envs/MLDFT37/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/hom

In [2]:
batch_size = 1
x_test = np.ones((batch_size,)+K.int_shape(x_in)[1:])
fn = K.function([x_in], [normal.x_normalized])
v1 = fn([x_test])
np.array(v1).shape ##(1,1,256)
#Out[7]: 
#array([0.08498713, 0.05334507, 0.        , 0.        , 0.07348377,
#       0.01111731, 0.06810687, 0.        , 0.        , 0.        ,
#       0.        , 0.1667348 , 0.        , 0.        , 0.07852139,

2024-09-13 18:07:14.015716: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2024-09-13 18:07:14.053974: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2900000000 Hz
2024-09-13 18:07:14.069581: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x559327c12550 executing computations on platform Host. Devices:
2024-09-13 18:07:14.069648: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2024-09-13 18:07:14.127356: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line flag, not via TF_XLA

(1, 1, 256)

In [3]:
batch_size = 1
outputs = [layer.output for layer in model.layers]          # all layer outputs
functors = [K.function([model.input], [out]) for out in outputs] # x_test: model.input

x_test = np.ones((batch_size,)+K.int_shape(x_in)[1:])
layer_outs = [func([x_test]) for func in functors]
for i in range(len(layer_outs)):
    print(np.array(layer_outs[i]).shape)
#Out[10]:
#(1, 1, 784)
#(1, 1, 512)
#(1, 1, 512)
#(1, 1, 256)
#(1, 1, 256)
#(1, 1, 256)
#(1, 1, 10)

InvalidArgumentError: input_1:0 is both fed and fetched.